In [1]:
import numpy as np
global_grid = None
"""
class FillList(list):
    def __setitem__(self, index, value):
        try:
            super().__setitem__(index, value)
        except IndexError:
            for _ in range(index-len(self)+1):
                self.append(None)
            super().__setitem__(index, value)
"""

def rectangular_grid_creation(n,m):
    new_grid = np.zeros(n,m)
    return rectangular_grid_creation

def set_global_grid(x):
    if x:
        global_grid = x
    else:
        raise ValueError("No grid Provided")
    return

def grid_extraction(grid):
    if grid == None:
        raise ValueError("No grid Provided")
    
    return np.shape(grid)
    
class Part_Library:

    def __init__(self):
        self.library = {}
    
    def add_to_library(self, part):
        if part ==None:
            raise ValueError("No Part Provided")
        
        part.check_rules()
        self.library[part.index] = part


#how are parts moved other than the rotations, like left right up and down.
class Part(Part_Library):

    def __init__(self, index, grid):
        super().__init__()
        self.index = [index if super.library[index] is None else None]
        self.grid = grid
        self.shapes = []

    def rotate_shape(self):
        #rotate about x direction -- in theory should be 16 total piece orientations
            #rotate about y axis
        return "Not Implemented"

    def part_setup(self):
        g = self.extract_shape()
        self.rotate_shape()
        return

    def extract_shape(self):
        g = grid_extraction(self.grid)
        #have to create something that extracts the shape of the grid



In [1]:
import numpy as np
from itertools import permutations, product
from typing import Iterable, List, Tuple, Dict

class Grid3D:
    """Finite 3‑D occupancy grid.

    Cells contain 0 for empty or a positive integer part‑id.
    Shape order is (X, Y, Z) ⇒ (width, depth, height)."""

    def __init__(self, dims: Tuple[int, int, int], dtype=np.int8):
        if any(d <= 0 for d in dims):
            raise ValueError("All grid dimensions must be positive integers.")
        self.shape: Tuple[int, int, int] = dims
        self.data: np.ndarray = np.zeros(self.shape, dtype=dtype)

    def inside(self, xyz: Tuple[int, int, int]) -> bool:
        """Return *True* if coordinate is within grid bounds."""
        x, y, z = xyz
        X, Y, Z = self.shape
        return 0 <= x < X and 0 <= y < Y and 0 <= z < Z

    def empty(self, xyz: Tuple[int, int, int]) -> bool:
        return self.data[xyz] == 0

    def can_place(self, coords: Iterable[Tuple[int, int, int]], origin: Tuple[int, int, int]) -> bool:
        """Validate that every translated *coord* is inside and empty."""
        ox, oy, oz = origin
        for dx, dy, dz in coords:
            pos = (ox + dx, oy + dy, oz + dz)
            if not (self.inside(pos) and self.empty(pos)):
                return False
        return True

    def place(self, coords: Iterable[Tuple[int, int, int]], origin: Tuple[int, int, int], part_id: int):
        if part_id <= 0:
            raise ValueError("part_id must be a positive integer.")
        if not self.can_place(coords, origin):
            raise ValueError("Cannot place – overlap or out‑of‑bounds.")
        ox, oy, oz = origin
        for dx, dy, dz in coords:
            self.data[ox + dx, oy + dy, oz + dz] = part_id

    def remove(self, coords: Iterable[Tuple[int, int, int]], origin: Tuple[int, int, int]):
        ox, oy, oz = origin
        for dx, dy, dz in coords:
            self.data[ox + dx, oy + dy, oz + dz] = 0

    def copy(self) -> "Grid3D":
        g = Grid3D(self.shape)
        g.data = self.data.copy()
        return g

    def filled(self) -> int:
        """Return number of occupied cells."""
        return int((self.data != 0).sum())

    def volume(self) -> int:
        X, Y, Z = self.shape
        return X * Y * Z

    def __repr__(self):
        return f"Grid3D(shape={self.shape}, filled={self.filled()}/{self.volume()})"


def _generate_rotation_matrices() -> List[np.ndarray]:
    rots = []
    for perm in permutations(range(3)):
        base = np.zeros((3, 3), dtype=int)
        for signs in product((-1, 1), repeat=3):
            for i, axis in enumerate(perm):
                base[i, axis] = signs[i]
            if np.linalg.det(base) == 1:
                rots.append(base.copy())
            base.fill(0)
    assert len(rots) == 24, "Expected 24 proper rotations"
    return rots

_ROT_MATS: List[np.ndarray] = _generate_rotation_matrices()

class Part:
    """A polycube defined by voxel coordinates relative to an origin (0,0,0)."""
    def __init__(self, pid: int, coords: Iterable[Tuple[int, int, int]]):
        self.id: int = pid
        self.base: Tuple[Tuple[int, int, int], ...] = tuple(coords)
        if not self.base:
            raise ValueError("Part must contain at least one voxel.")
        self.orientations: List[Tuple[Tuple[int, int, int], ...]] = self._unique_orientations()

    def _unique_orientations(self) -> List[Tuple[Tuple[int, int, int], ...]]:
        """Return list of canonicalised orientations (translated so min‑coord = 0)."""
        unique = {
            self._normalise(tuple((r @ np.array(v)).tolist() for v in self.base))
            for r in _ROT_MATS
        }
        return sorted(unique)

    @staticmethod
    def _normalise(coords: Tuple[Tuple[int, int, int], ...]) -> Tuple[Tuple[int, int, int], ...]:
        arr = np.array(coords)
        min_vals = arr.min(axis=0)
        norm = tuple(map(tuple, arr - min_vals))
        return tuple(sorted(norm))

    def size(self) -> int:
        return len(self.base)

    def __repr__(self):
        return f"Part(id={self.id}, size={self.size()}, orientations={len(self.orientations)})"

class PartLibrary:
    def __init__(self):
        self._parts: Dict[int, Part] = {}

    def add(self, part: Part):
        if part.id in self._parts:
            raise ValueError(f"Duplicate part id {part.id}")
        self._parts[part.id] = part

    def __getitem__(self, pid: int) -> Part:
        return self._parts[pid]

    def values(self) -> List[Part]:
        return list(self._parts.values())

    def __iter__(self):
        return iter(self._parts.values())

    @classmethod
    def from_dict(cls, d: Dict[str, List[List[int]]]):
        """Expect {"1": [[0,0,0], …], "2": …}."""
        lib = cls()
        for key, vox in d.items():
            lib.add(Part(int(key), [tuple(v) for v in vox]))
        return lib

if __name__ == "__main__":
    board = Grid3D((3, 3, 3))
    print(board)

    v_piece = Part(1, [(0, 0, 1), (1, 0, 0), (0, 1, 0)])
    print(v_piece)
    print("Unique orientations:", len(v_piece.orientations))
    


Grid3D(shape=(3, 3, 3), filled=0/27)
Part(id=1, size=3, orientations=8)
Unique orientations: 8
